In [1]:
import pandas as pd
from sklearn.linear_model import Ridge, LinearRegression
import numpy as np
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

In [13]:
all_factors = pd.read_csv('jhjj_data/jhjj_factor_2022-07-05.csv')
all_factors = all_factors.rename(columns={'dataDate': 'tradeDate'})
all_factors = all_factors.drop(columns='Unnamed: 0')
all_factors['tradeDate'] = all_factors.tradeDate.apply(lambda x: str(x).replace('-', ''))
all_factors

,ticker,tradeDate,volume,cachgPct,thecommittee,askVolume1,bidVolume1,OCVP1,Gap,Open/vwap-1,caTr,caQrr
0,600000,20220705,119142,-0.001253,-0.278412,236285,17158.0,0.005588,-0.003750,0.000125,0.000405,0.688284
1,600004,20220705,38100,-0.004270,-0.700108,192,3100.0,0.001676,-0.035172,-0.009137,0.001603,0.313839
2,600006,20220705,217700,-0.004155,-0.678207,19300,4100.0,0.003649,-0.027064,-0.009096,0.010840,1.041129
3,600007,20220705,3100,-0.007717,-0.154472,1100,500.0,0.000725,-0.025268,-0.000389,0.000305,0.143519
4,600008,20220705,242700,0.000000,-0.248042,1467150,972000.0,0.004258,-0.006826,0.000000,0.003306,1.097197
...,...,...,...,...,...,...,...,...,...,...,...,...
4367,301286,20220705,26700,-0.000431,0.413333,900,500.0,0.005277,-0.012766,0.005853,0.006670,0.911263
4368,301288,20220705,19700,-0.017699,-0.043478,800,800.0,0.005644,-0.041977,-0.019356,0.017916,1.304636
4369,301289,20220705,9600,-0.004149,-0.573333,3500,1600.0,0.004716,-0.038290,-0.013203,0.015934,1.090909
4370,301298,20220705,35600,-0.010878,0.576471,300,1500.0,0.007454,-0.044003,-0.020664,0.023987,6.592593


In [18]:
all_factors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4372 entries, 0 to 4371
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ticker        4372 non-null   int64  
 1   tradeDate     4372 non-null   object 
 2   volume        4372 non-null   int64  
 3   cachgPct      4372 non-null   float64
 4   thecommittee  4372 non-null   float64
 5   askVolume1    4372 non-null   int64  
 6   bidVolume1    4372 non-null   float64
 7   OCVP1         4372 non-null   float64
 8   Gap           4372 non-null   float64
 9   Open/vwap-1   4372 non-null   float64
 10  caTr          4372 non-null   float64
 11  caQrr         4372 non-null   float64
dtypes: float64(8), int64(3), object(1)
memory usage: 410.0+ KB


In [15]:
prediction = pd.read_csv('prediction_results/20220705.csv')
prediction = prediction.rename(columns={'date': 'tradeDate'})
prediction = prediction.drop(columns='Unnamed: 0')
prediction['ticker'] = prediction['ticker'] .astype(np.int64)
prediction

,ticker,tradeDate,prediction
0,1,20220705,0.000323
1,10,20220705,0.000982
2,100,20220705,0.000406
3,11,20220705,0.001240
4,12,20220705,0.000300
...,...,...,...
4676,995,20220705,0.000368
4677,996,20220705,0.000614
4678,997,20220705,0.001214
4679,998,20220705,-0.000811


In [19]:
prediction['tradeDate'] = prediction.tradeDate.apply(lambda x: str(x))
prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4681 entries, 0 to 4680
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ticker      4681 non-null   int64  
 1   tradeDate   4681 non-null   object 
 2   prediction  4681 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 109.8+ KB


In [20]:
all_factors_p = all_factors.merge(prediction, on=['ticker', 'tradeDate'], how='left')

In [23]:
all_factors_p = all_factors_p[~all_factors_p.prediction.isnull()]
all_factors_p.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4299 entries, 0 to 4363
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ticker        4299 non-null   int64  
 1   tradeDate     4299 non-null   object 
 2   volume        4299 non-null   int64  
 3   cachgPct      4299 non-null   float64
 4   thecommittee  4299 non-null   float64
 5   askVolume1    4299 non-null   int64  
 6   bidVolume1    4299 non-null   float64
 7   OCVP1         4299 non-null   float64
 8   Gap           4299 non-null   float64
 9   Open/vwap-1   4299 non-null   float64
 10  caTr          4299 non-null   float64
 11  caQrr         4299 non-null   float64
 12  prediction    4299 non-null   float64
dtypes: float64(9), int64(3), object(1)
memory usage: 470.2+ KB


In [25]:
f_x = ['volume', 'cachgPct', 'thecommittee', 'askVolume1',
        'bidVolume1', 'caQrr', 'caTr', 'OCVP1',
        'Open/vwap-1', 'Gap', 'prediction']

In [26]:
scaler = StandardScaler()
all_factors_p[f_x] = scaler.fit_transform(all_factors_p[f_x])

/workspace1/liufengyuan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/workspace1/liufengyuan/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [27]:
all_factors_p['linear_com'] = all_factors_p[['volume', 'cachgPct', 'thecommittee', 'askVolume1',
        'bidVolume1', 'caQrr', 'caTr', 'OCVP1',
        'Open/vwap-1', 'Gap']].sum(axis=1)/20 + all_factors_p['prediction']/2
all_factors_p

/workspace1/liufengyuan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,ticker,tradeDate,volume,cachgPct,thecommittee,askVolume1,bidVolume1,OCVP1,Gap,Open/vwap-1,caTr,caQrr,prediction,linear_com
0,600000,20220705,-0.065950,0.147984,-0.562667,0.056839,-0.034227,-0.022758,1.141331,0.246599,-0.243058,-0.038761,0.481806,0.272170
1,600004,20220705,-0.143901,-0.134219,-1.453078,-0.043512,-0.037060,-0.042509,-0.357033,-0.323006,-0.225765,-0.042183,-0.147912,-0.214069
2,600006,20220705,0.028849,-0.123432,-1.406836,-0.035391,-0.036859,-0.032545,0.029632,-0.320503,-0.092387,-0.035536,-0.673059,-0.437780
3,600007,20220705,-0.177566,-0.456569,-0.300967,-0.043126,-0.037584,-0.047311,0.115232,0.214977,-0.244502,-0.043740,0.204547,0.051216
4,600008,20220705,0.052896,0.265186,-0.498541,0.580018,0.158246,-0.029475,0.994656,0.238882,-0.201170,-0.035024,0.402010,0.277289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4354,301236,20220705,-0.167484,0.529056,0.895938,-0.041129,-0.034376,-0.033398,-0.083190,-0.122323,-0.217954,-0.018640,0.005078,0.037864
4355,301237,20220705,-0.177470,-0.511333,0.519381,-0.043084,-0.037484,-0.039619,-0.849738,0.015158,-0.191632,-0.041485,-0.707025,-0.421378
4359,301256,20220705,-0.099751,-0.629813,1.857144,-0.043381,-0.032968,-0.024735,-1.497576,-1.063903,0.001368,-0.037177,-0.365848,-0.261464
4361,301258,20220705,-0.178816,-0.581118,-0.308195,-0.043551,-0.037665,-0.047678,-0.680079,-0.502269,-0.220815,-0.042481,-0.024171,-0.144219


In [75]:
result = all_factors_p[['ticker', 'tradeDate', 'linear_com']]
result

,ticker,tradeDate,linear_com
0,600000,20220705,0.272170
1,600004,20220705,-0.214069
2,600006,20220705,-0.437780
3,600007,20220705,0.051216
4,600008,20220705,0.277289
...,...,...,...
4354,301236,20220705,0.037864
4355,301237,20220705,-0.421378
4359,301256,20220705,-0.261464
4361,301258,20220705,-0.144219


In [74]:
result = result.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
result

,ticker,tradeDate,linear_com,prediction
0,1,20220705,0.195249,0.272170
1,2,20220705,-0.390879,-0.214069
2,4,20220705,0.469451,-0.437780
3,5,20220705,-0.115491,0.051216
4,6,20220705,0.974618,0.277289
...,...,...,...,...
4294,688799,20220705,-0.545025,-0.392247
4295,688800,20220705,-0.974352,-0.972628
4296,688819,20220705,-0.923787,-0.005976
4297,688981,20220705,-0.731960,0.136437


In [31]:
f_x = ['volume', 'cachgPct', 'thecommittee', 'askVolume1',
        'bidVolume1', 'caQrr', 'caTr', 'OCVP1', 'Open/vwap-1', 'Gap']
f_index = ['ticker', 'tradeDate']

scaler = StandardScaler()
all_factors_p[f_x] = scaler.fit_transform(all_factors_p[f_x])

result = all_factors_p[f_index]
result['linear_com'] = all_factors_p[f_x].sum(axis=1)/20 + all_factors_p['prediction']/2
result = result.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
result

/workspace1/liufengyuan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/workspace1/liufengyuan/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/workspace1/liufengyuan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

,ticker,tradeDate,linear_com
0,1,20220705,0.195249
1,2,20220705,-0.390879
2,4,20220705,0.469451
3,5,20220705,-0.115491
4,6,20220705,0.974618
...,...,...,...
4294,688799,20220705,-0.545025
4295,688800,20220705,-0.974352
4296,688819,20220705,-0.923787
4297,688981,20220705,-0.731960


In [72]:
all_factors_p.loc[:, f_x] = scaler.fit_transform(all_factors_p.loc[:, f_x])

/workspace1/liufengyuan/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [76]:
result.loc[:, 'prediction'] = all_factors_p.loc[:, f_x].sum(axis=1)/20 + all_factors_p.loc[:, 'prediction']/2

/workspace1/liufengyuan/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/workspace1/liufengyuan/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [37]:
import os
import sys
import glob

In [38]:
from datetime import date,timedelta

In [46]:
import time
import datetime

today = datetime.date.today()
today

datetime.date(2022, 7, 6)

In [67]:
today = datetime.datetime(2022, 7, 12)
if today.isoweekday() == 1:
    today -= datetime.timedelta(days= 3)
elif today.isoweekday() in set((2, 3, 4, 5)):
    today -= datetime.timedelta(days= 1)
else:
    today -= datetime.timedelta(days = today.isoweekday() % 5)
today

datetime.datetime(2022, 7, 11, 0, 0)

In [57]:
filename1 = glob.glob('jhjj_data/' + today.strftime("%Y-%m-%d") + '.csv')
filename2 = glob.glob('prediction_results/' + today.strftime("%Y%m%d") + '.csv')